In [6]:
%load_ext autoreload
%autoreload 2
from app.config_loader import load_config_once
conf = load_config_once(output_folder="SIDKIM")
import pdfplumber, camelot, pprint, json5,json, os, re
import fitz #type: ignore
import pandas as pd
import numpy as np

from app.fund_sid_data import *

with open("config\\sid_params.json5","r+") as file:
    params = json5.load(file)

# with open("config\\sid_params.json5","r+") as file:
#     params = json5.dump(params,file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Config already loaded. Skipping re-initialization.


In [ ]:
path = os.path.join(os.getcwd(),"pdfs","96_17078_Apr-2025_1746183006_SID.pdf")
object = AngelOne(amc_id="96",path=path)
temp_dict = {}
temp_dict.update(object._parse_page_zero([0]))
temp_dict.update(object._parse_scheme_table_data("5,6,7,8,9,10"))
temp_dict.update(object._parse_fund_manager_info("16,17"))
# object = AngelOne(amc_id="96",path=path)
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)
with open("tempor.json","w+") as file:
    json.dump(dfs,file)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\DSP Mutual Fund\\8_17162_May-2025_1748246532_SID.pdf")
object = DSP(amc_id="8", path=path)
temp_dict = {}
temp_dict.update(object._parse_page_zero([0]))
temp_dict.update(object._parse_scheme_table_data("5,6,7,8"))
temp_dict.update(object._parse_fund_manager_info("21,22"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)
# with open("tempor.json","w+") as file:
#     json.dump(dfs,file)


In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Tata Mutual Fund\\38_17082_May-2025_1746423571_SID.pdf")
object = Tata(amc_id="38", path=path)
temp_dict = {}
temp_dict.update(object._parse_page_zero([0]))
temp_dict.update(object._parse_scheme_table_data("3,4,5"))
temp_dict.update(object._parse_fund_manager_info("12"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [7]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Nippon India Mutual Fund\\33_17142_May-2025_1747631690_SID.pdf")
object = Nippon(amc_id="33", path=path)
temp_dict = {}
temp_dict.update(object._parse_page_zero([0]))
temp_dict.update(object._parse_scheme_table_data("4,5,6"))
temp_dict.update(object._parse_fund_manager_info("11"))

Function Running: _parse_page_zero
Function Running: _parse_scheme_table_data


c:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\app\parse_sid_pdf.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series


Function Running: _parse_fund_manager_info


c:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\app\parse_sid_pdf.py:161: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)


In [10]:
object = Nippon(amc_id="33", path=path)
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

Function Running: refine_extracted_data
Function Running: merge_and_select_data
[re.compile('.*offer_of_units$', re.IGNORECASE)]


In [11]:
with open("temp.json","w+") as file:
    json.dump(temp_dict,file)
with open("data.json","w+") as file:
    json.dump(data,file)
with open("dfs.json","w+") as file:
    json.dump(dfs,file)

In [ ]:
# def _parse_page_zero(path:str,amc:str,pages:list):
#     final_dict = {}
#     bbox_text = []
#     other_text = []
#     risk_bbox = params[amc]["PARAMS"]["page_zero"]["bbox"]
#     with fitz.open(path) as doc:
#         for pgn, page in enumerate(doc):
#             if pgn in pages:
#                 bbox_text.append(page.get_text("text", clip=risk_bbox).replace("\n", ""))
#                 other_text.append(page.get_text("text").replace("\n", ""))
#         final_dict["risk_bbox"] = bbox_text
#         final_dict["other_text"] = other_text
#     return final_dict

import fitz  # PyMuPDF
import re

def _parse_page_zero(path: str, amc: str, pages: list) -> dict:
    final_dict = {"risk_bbox": [], "other_text": []}
    risk_bbox = params[amc]["PARAMS"]["page_zero"]["bbox"]

    with fitz.open(path) as doc:
        for pgn in pages:
            page = doc[pgn]

            def extract_sorted_text_blocks(clip_area=None):
                blocks = page.get_text("dict", clip=clip_area)["blocks"]
                text_blocks = []

                for block in blocks:
                    if block.get("type") == 0:
                        text = ""
                        for line in block.get("lines", []):
                            for span in line.get("spans", []):
                                text += span.get("text", "")
                        text = re.sub(r"\s+", " ", text).strip()
                        if text:
                            text_blocks.append((block["bbox"], text))

                return [text for _, text in sorted(text_blocks, key=lambda b: (b[0][1], b[0][0]))]

            # Extract bbox text
            final_dict["risk_bbox"].extend(extract_sorted_text_blocks(clip_area=risk_bbox))

            # Extract full text
            final_dict["other_text"].extend(extract_sorted_text_blocks())

    return final_dict

def _detect_column_start_by_keywords(df, threshold, keywords):
    normalized_keywords = [re.sub(r"\s+", " ", kw.strip().lower()) for kw in keywords]

    for i in range(df.shape[1]):
        col = df.iloc[:, i].dropna().astype(str)
        col_cleaned = col.apply(lambda x: re.sub(r"\s+", " ", x).strip().lower())
        col_cleaned = col_cleaned[col_cleaned != ""]

        count = sum(any(kw in cell for kw in normalized_keywords) for cell in col_cleaned)

        if count >= threshold:
            return i
    return 1

def _parse_scheme_table_data(path:str,amc:str,pages:str):
    final_dict = {}
    
    threshold = params[amc]["PARAMS"]["table_data"]["threshold"]
    keywords = params[amc]["PARAMS"]["table_data"]["keywords"]
    # print(keywords)
    
    scheme_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    
    dfs = pd.concat([table.df for table in scheme_tables], ignore_index=True)
    col_start = _detect_column_start_by_keywords(dfs,threshold=threshold,keywords=keywords)
    # print(col_start)
    
    dfs = dfs.iloc[:,col_start:]
    # print(dfs.head(20))
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series
    dfs.iloc[:, 0] = dfs.iloc[:, 0].replace(r'^\s*$', np.nan, regex=True)
    dfs.iloc[:, 0] = dfs.iloc[:, 0].ffill() #fill corresponding structure
    dfs.columns = ['Title'] + [f'Data{i}' for i in range(1, dfs.shape[1])] #new col structure
    
    data_cols = [col for col in dfs.columns if col != 'Title']
    for title, group_df in dfs.groupby('Title', sort=False):
        title = re.sub(r"[^A-Za-z0-9\s]+","",title).strip()
        title = re.sub(r"\s+"," ",title).strip()
        title = "_".join(title.lower().split(" "))
        
        current_title_values = []
        for index, row in group_df.iterrows():
            for col_name in data_cols:
                cell_value = row[col_name]
                if isinstance(cell_value, str) and cell_value.strip() != '':
                    current_title_values.append(cell_value)
        final_dict[title] = current_title_values
        
    return final_dict

def _detect_manager_column_start_by_keywords(df, threshold, keywords):
    normalized_keywords = [re.sub(r"\s+", " ", kw.strip().lower()) for kw in keywords]
    first_keyword = normalized_keywords[0]

    for i in range(df.shape[1]):
        col = df.iloc[:, i].dropna().astype(str)
        col_cleaned = col.apply(lambda x: re.sub(r"\s+", " ", x).strip().lower())
        col_cleaned = col_cleaned[col_cleaned != ""]

        count = sum(any(kw in cell for kw in normalized_keywords) for cell in col_cleaned)

        if count >= threshold:
            return i
    return 1

def _parse_fund_manager_info(path:str,amc:str,pages:str):
    
    row_count = params[amc]["PARAMS"]["manager_data"]["row_count"]
    match_order = params[amc]["PARAMS"]["manager_data"]["order"]
    print(row_count, match_order)
    manager_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    dfs = pd.concat([table.df for table in manager_tables], ignore_index=True)
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
    
    data_rows = [list(row) for _, row in dfs.iterrows() if row.count()==row_count]
    manager_list = []
    for data_r in data_rows:
        manager_list.append({
            "name":data_r[match_order["name"]],
            "experience":data_r[match_order["experience"]],
            "qualification":data_r[match_order["qualification"]],
        })
    return dfs, {"fund_manager": manager_list}

In [15]:
path = os.path.join(os.getcwd(),"pdfs",r"May SID\Tata Mutual Fund\38_17082_May-2025_1746423571_SID.pdf")
# df = _parse_scheme_table_data(path,"38","3,4,5")
# df = _parse_page_zero(path,"38",[0])
dfs,df = _parse_fund_manager_info(path,"38","12")

6 {'name': 0, 'experience': 3, 'qualification': 2}


C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_13264\2172782239.py:107: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)


In [16]:
dfs.head(20)

,0,1,2,3,4,5,6,7
0,T P P S SID – TATA INCOME PLUS ARBITRAGE...,,,,,,,
1,,Name,Age,Qualification,Total Experience (in years),Other Schemes Under His Management,Experience (Assignments held during last 10 ye...,
2,,Sailesh Jain (Managing Since: Not applica...,45,MBA (Finance),22,"Tata Arbitrage Fund, Tata Balanced Advantag...",F F F From Nov’2018 with Tata Asset M...,
3,,Abhishek Sonthalia (Co- Managing Since: N...,40,"CFA Charter holder, management programme f...",16,"Tata Corporate Bond Fund, Tata Liquid Fund, ...",E F F Mr. Abhishek Sonthalia is Fund Manag...,
4,,,,,,,,


In [10]:
pprint.pprint(df)

{'fund_manager': []}


In [ ]:
temp_dict = {}
temp_dict.update(_parse_page_zero(path,"38",[0]))
temp_dict.update(_parse_scheme_table_data(path,"38","3,4,5"))
temp_dict.update(_parse_fund_manager_info(path,"38","29,30"))

In [162]:
with open("tempor.json","w+") as file:
    json.dump(temp_dict,file)